[Hyun woo kim] - 2018-10-19

![](https://storage.googleapis.com/kaggle-media/competitions/PUBG/PUBG%20Inlay.jpg)

This kernel is the changed [PUBG Data Description A to Z + FE with Python kernel](https://www.kaggle.com/chocozzz/pubg-data-description-a-to-z-fe-with-python) due to data leakage.

## Notebook Outline
- Variable Description ( what is different before )

In [ ]:
import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

## Variable Description ( what is different before )

### Name of Train, test file changed : 
- train.csv -> train_V2.csv
- test.csv -> test_V2.csv

In [ ]:
df_train = pd.read_csv('../input/train_V2.csv')
df_test  = pd.read_csv('../input/test_V2.csv')

In [ ]:
print("Train : ",df_train.shape)
print("Test : ",df_test.shape)

### Shape of Train, test changed :
- Train :  (4357336, 26) -> (4446966, 29)
- Test :  (1867913, 25) -> (1934174, 28)

data size increase little. Perhaps we will experience memory errors again.

### What's features updated ?? 

MichaelApers metioned : https://www.kaggle.com/c/pubg-finish-placement-prediction/discussion/68965
- `matchDuration` : Duration of match in seconds.  **( I think game time not loading time)**
- `matchType` : String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches.
- `rankPoints` : Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.


### Id changed to secret code

In [ ]:
df_train.head()

- Before Id, groupId, matchId 
![](https://github.com/choco9966/Team-EDA/blob/master/image/before.PNG?raw=true)

### Match Duration

This variable indicates how long the game took. So important variables affecting the target variable (like `walkDistance`) must be weighted according to Match Duration

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(18, 8))
sns.distplot(df_train['matchDuration'])

- The interesting thing is that there are two peaks. I think the game mode(match type) will be affected.
- 800 ~ 1000 very little peaks

In [ ]:
plt.figure(figsize = (18, 8))

sns.kdeplot(df_train.loc[df_train['matchType'] == 'squad-fpp', 'matchDuration'] , label = 'squad-fpp')

sns.kdeplot(df_train.loc[df_train['matchType'] == 'duo-fpp', 'matchDuration'] , label = 'duo-fpp')

sns.kdeplot(df_train.loc[df_train['matchType'] == 'squad', 'matchDuration'] , label = 'squad')

sns.kdeplot(df_train.loc[df_train['matchType'] == 'solo-fpp', 'matchDuration'] , label = 'solo-fpp')

sns.kdeplot(df_train.loc[df_train['matchType'] == 'duo', 'matchDuration'] , label = 'duo')

sns.kdeplot(df_train.loc[df_train['matchType'] == 'solo', 'matchDuration'] , label = 'solo')

# Labeling of plot
plt.xlabel('matchDuration'); plt.ylabel('Density'); plt.title('Distribution of matchDuration with Basic mode');

In [ ]:
plt.figure(figsize = (18, 8))

sns.kdeplot(df_train.loc[df_train['matchType'] == 'crashfpp', 'matchDuration'] , label = 'crashfpp')

sns.kdeplot(df_train.loc[df_train['matchType'] == 'crashtpp', 'matchDuration'] , label = 'crashtpp')

sns.kdeplot(df_train.loc[df_train['matchType'] == 'flarefpp', 'matchDuration'] , label = 'flarefpp')

sns.kdeplot(df_train.loc[df_train['matchType'] == 'flaretpp', 'matchDuration'] , label = 'flaretpp')


# Labeling of plot
plt.xlabel('matchDuration'); plt.ylabel('Density'); plt.title('Distribution of matchDuration with Event mode');

In crash mode, game time is generally shorter. So, if you look at the histogram above, you can see 800 ~ 1000 protruding a little.

### matchType

In [ ]:
f, ax = plt.subplots(figsize=(8, 6))
df_train['matchType'].value_counts().sort_values(ascending=False).plot.bar()
plt.show()

In [ ]:
print("Unique value of matchType :",len(df_train['matchType'].unique()))

- `fpp` : FPP stands for first-person perspective. Unless you’ve barely picked up a joypad or controller in your life, there’s a good chance you know what a first-person perspective looks like in a video game, and it’s the exact same proposition in PUBG. Of course, it’s a little more complicated than that. While there are FPP-only PUBG modes, away from that, there’s a raging debate over what works best: FPP or TPP (third-person perspective). But first: the FPP-only modes.

- `tpp` : Third person perspective. 


When you are playing a game with a third person, you can see the part that was not seen at fpp, like the image below or the below video.

![](https://t1.daumcdn.net/cfile/tistory/998C61405A77501315)

- This is a scene where the person on the right sees the red person in front of the door as a third person perspective

you see the video in https://www.youtube.com/watch?v=58nG88nGp80 . 

original picture resource in http://bird-of-prey.tistory.com/2

Game mode Descrition : https://pubg.gamepedia.com/Game_Modes 

### Basic mode
- **`sqaud`** : Squad is a gameplay type where players are organized into teams which are then pitted against each other. This is distinct from the classic Free For All (FFA) gameplay mode, where players are free to kill whomever they wish.SQUAD, a game mode where you can team up in groups of 2, 3 or 4 players, or if you prefer, you can still play solo and take on everyone alone in the match.  

![](https://d1u5p3l4wpay3k.cloudfront.net/battlegrounds_gamepedia_en/thumb/3/38/Game_Mode_Group_Play.jpg/400px-Game_Mode_Group_Play.jpg?version=da31367e5836ba1ed4676915269c2fc6)

Note: No matter the size of your party when you enter a match, you'll be matched with teams of 4.

- **`sqaud-fpp`** : Squad FPP(First Person Perspective) is based around your normal Squad game mode (as read above), but instead of being in 3rd person this is strictly in 1st person.

- **`duo`** : For this game mode, you will be paired up with another individual and will compete to be the last ones alive.

- **`duo-fpp`** : Duo FPP is based around your normal Duo game mode (as read above), but instead of being in 3rd person this is strictly in 1st person.

- **`solo`** : The classic game mode. Complete free for all, kill everyone, be the last one alive. Solo, a game mode where you spawn into the world alone and you just rely on your own tactics and skill to push you to the end and be the last player alive.

- **`solo-fpp`** : Solo FPP is based around your normal Solo game mode (as read above), but instead of being in 3rd person this is strictly in 1st person.


### Event mode
- **`flare (flare gun)`** : Flare Gun can be obtained. A flare gun is a kind of signal that shoots toward the sky. It is a special firearm that supplies a special package (Care Package) when the airplane is deployed to that position when in use.

![](https://github.com/choco9966/Team-EDA/blob/master/image/flare.jpg?raw=true)


- **`crash (Crash Carnage)`** : " Road scary is the subject of this week's event mode, where fuel and fire are king." In Crashcanage, there are no firearms, so melee weapons, drones, You have to concentrate on driving skills that can move the Duo to the final round. In this event, the circle moves quite fast, so it quickly crashes on the way to the loot, the vehicle, and the warrior glory of the road. " i.e. It means to win only with a melee weapon without a gun.

![](http://gamove.ru/uploads/thumbs/412161970-1.jpg)



see the video : 
- https://www.youtube.com/watch?v=meLJX0-2YLY
- https://www.youtube.com/watch?v=5rbIhg6mNPI

https://comicbook.com/gaming/2018/10/04/pubg-event-mode-crash-carnage-/

### rankPoints
- `rankPoints` : Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.

In [ ]:
#histogram
f, ax = plt.subplots(figsize=(18, 8))
sns.distplot(df_train['rankPoints'])

In [ ]:
print("rankPoints has -1 :", len(df_train[df_train['rankPoints'] == -1])/df_train.shape[0])
print("rankPoints has 0 :", len(df_train[df_train['rankPoints'] == 0])/df_train.shape[0])
print("rankPoints has others :", len(df_train[(df_train['rankPoints'] != 0) & (df_train['rankPoints'] != -1)])/df_train.shape[0])

In [ ]:
f, ax = plt.subplots(figsize=(18, 8))
df_train[df_train['rankPoints'] == -1]['matchType'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('Count of matchType with rankPoints == -1');
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(18, 8))
df_train[(df_train['rankPoints'] != 0) & (df_train['rankPoints'] != -1)]['matchType'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('Count of matchType with rankPoints != -1 & != 0');
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(18, 8))
df_train[df_train['rankPoints'] == 0]['matchType'].value_counts().sort_values(ascending=False).plot.bar()
plt.title('Count of matchType with rankPoints == 0');
plt.show()


There is no difference in the frequency of game types according to rankpoints. but
- rankPoints == -1 is no play game mode `flarefpp`, `crashfpp`, `crashtpp`
- rankPoints == 0 is no play game mode `flarefpp`, `crashfpp`, `crashtpp`, `flaretpp` ,`normal-duo`,`normal-squad`

I have some questions :

Q1. what is normal squad fpp??
look the above picture. Squad is divided into three. 
- squad 
- normal squad fpp 
- squad fpp 
i know squad and squad fpp. but what is normal squad fpp ???

Q2. TPP vs fpp ?  
It is only tpp mode with tpp in type name, only fpp mode with fpp in type name, and tpp and fpp can changed if it is not(like squad)?

### I'll find more resources and update the kernel.